In [172]:
# get the previous day's fantasy results from FantasyData
import requests
import pandas as pd

# get yesterday's date, convert to string
from datetime import date, timedelta
date = date.today() - timedelta(1)
datestr = date.strftime('%m-%d-%Y')

import requests
session_requests = requests.session()

headers = {
    'authority': 'fantasydata.com',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-language': 'en-US,en;q=0.9',
    'accept-encoding': 'gzip, deflate',
    'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
}

data = {
    'sort': 'Name-asc',
    'pageSize': '300',
    'group': '',
    'filter': '',
    'filters.scope': '2',
    'filters.subscope': '',
    'filters.season': '2023',
    'filters.seasontype': '1',
    'filters.team': '',
    'filters.conference': '1',
    'filters.position': '',
    'filters.searchtext': '',
    'filters.scoringsystem': '',
    'filters.exportType': '',
    'filters.date': datestr,
    'filters.dfsoperator': '',
    'filters.dfsslateid': '',
    'filters.dfsslategameid': '',
    'filters.dfsrosterslot': '',
    'filters.showfavs': '',
    'filters.teamkey': '',
    'filters.oddsstate': '',
    'filters.showall': '',
}

resp = session_requests.get('https://fantasydata.com/NBA_FantasyStats/FantasyStats_Read', headers=headers, data=data)
print(resp)
data = resp.json() #Grab Data In JSON format
df = pd.json_normalize(data, record_path=['Data']) #Transform Json to dataframe

<Response [200]>


In [173]:
df

,StatID,TeamID,PlayerID,Name,Team,Position,Games,Started,Season,Points,...,Game.AwayTeamScore,Game.Quarter,Game.Status,Game.IsOver,Game.ShouldHaveStarted,Game.TimeRemainingMinutes,Game.TimeRemainingSeconds,Game.DateTime,Game.TimeRemaining,Game.QuarterDisplay
0,1061172,25,20002725,A.J. Lawson,DAL,SG,0,0,2023,0.0,...,99,None,Final,True,True,None,None,/Date(1674788400000)/,,F
1,1061076,9,20000603,Al Horford,BOS,C,1,1,2023,6.0,...,120,None,F/OT,True,True,None,None,/Date(1674779400000)/,,F/OT
2,1061023,14,20000837,Alec Burks,DET,SG,1,1,2023,20.0,...,130,None,Final,True,True,None,None,/Date(1674779400000)/,,F
3,1061089,11,20001801,Alex Caruso,CHI,SG,1,0,2023,2.0,...,96,None,Final,True,True,None,None,/Date(1674779400000)/,,F
4,1061146,22,20002666,Alperen Sengün,HOU,C,1,1,2023,14.0,...,113,None,Final,True,True,None,None,/Date(1674781200000)/,,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,1061148,22,20002882,TyTy Washington Jr.,HOU,PG,1,0,2023,10.0,...,113,None,Final,True,True,None,None,/Date(1674781200000)/,,F
195,1061140,22,20002675,Usman Garuba,HOU,PF,0,0,2023,0.0,...,113,None,Final,True,True,None,None,/Date(1674781200000)/,,F
196,1061050,8,20002058,Yuta Watanabe,BKN,PF,1,0,2023,8.0,...,130,None,Final,True,True,None,None,/Date(1674779400000)/,,F
197,1061193,24,20001834,Zach Collins,SA,C,1,0,2023,7.0,...,100,None,Final,True,True,None,None,/Date(1674790200000)/,,F


In [174]:
# create dataframe with yesterday's results
results_df = df[['Name', 'FantasyPointsFanDuel']]

In [175]:
# read in model predictions

# change formatting of date string to read in predictions
date = date.today() - timedelta(1)
datestr = date.strftime('%Y%m%d')

import os

working_directory = 'D:/machine_learning/DFS/NBA/NBA_moredata'
os.chdir(working_directory)
pred_dir = 'Predictions/'

# read in model predictions
g_preds = pd.read_csv(pred_dir + 'G_Predictions_RF_'+str(datestr)+'.csv', index_col = 0)
f_preds = pd.read_csv(pred_dir + 'F_Predictions_RF_'+str(datestr)+'.csv', index_col = 0)
c_preds = pd.read_csv(pred_dir + 'C_Predictions_RF_'+str(datestr)+'.csv', index_col = 0)

# concat predictions to a single dataframe
all_preds = pd.concat([g_preds, f_preds, c_preds]).reset_index(drop = True)

# fix names with accent marks/suffixes
import unidecode

for i in range(len(all_preds)):
    # accent mark fix
    all_preds.iloc[i, 2] = unidecode.unidecode(all_preds.iloc[i, 2])
    # drop Jrs.
    if all_preds.iloc[i, 2][-3:] == 'Jr.' or all_preds.iloc[i, 2][-2:] == 'IV':
        all_preds.iloc[i, 2] = all_preds.iloc[i, 2][:-3]

In [176]:
# drop lower prediction for players whose performance was predicted with multiple models
all_preds = all_preds.sort_values('Prediction_RF', ascending = False).drop_duplicates(subset = 'Player')

# rename Player column to Name
all_preds = all_preds.rename({'Player':'Name'}, axis = 1)

In [177]:
results_df = results_df.merge(all_preds, on = 'Name')

In [178]:
from sklearn.metrics import mean_absolute_error
print("{:.2f}".format(mean_absolute_error(results_df['FantasyPointsFanDuel'], results_df['Prediction_RF'])))

12.24


In [179]:
results_df.head(n=50)

,Name,FantasyPointsFanDuel,Season,Date,Team,Opp,Prediction_RF,PredictRank_RF
0,A.J. Lawson,0.0,2022,20230126,DAL,PHX,12.9598,108.0
1,Al Horford,14.8,2022,20230126,BOS,NYK,28.6415,12.0
2,Alec Burks,35.0,2022,20230126,DET,BRK,22.7917,40.0
3,Alex Caruso,12.6,2022,20230126,CHI,CHO,21.0728,43.0
4,Amir Coffey,4.7,2022,20230126,LAC,SAS,14.0976,91.0
5,Andre Drummond,0.0,2022,20230126,CHI,CHO,14.2742,26.0
6,Ayo Dosunmu,20.8,2022,20230126,CHI,CHO,22.4332,41.0
7,Ben Simmons,11.7,2022,20230126,BRK,DET,31.3682,19.0
8,Bismack Biyombo,12.1,2022,20230126,PHO,DAL,24.8364,17.0
9,Blake Griffin,0.0,2022,20230126,BOS,NYK,11.8038,31.0
